In [9]:
# https://www.geeksforgeeks.org/strassens-matrix-multiplication/

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Stranssen's Algorithm

In [10]:
def create_matrices(n):
    # Criar duas matrizes de inteiros com dimensão n x n
    matrix1 = [[random.randint(0, 100) for _ in range(n)] for _ in range(n)]
    matrix2 = [[random.randint(0, 100) for _ in range(n)] for _ in range(n)]
    
    return matrix1, matrix2

#Function to add two matrices
def add_matrix(matrix_A, matrix_B, matrix_C, split_index): 
    for i in range(split_index): 
        for j in range(split_index): 
            matrix_C[i][j] = matrix_A[i][j] + matrix_B[i][j] 
            
def multiply_matrix(matrix_A, matrix_B): 
    col_1 = len(matrix_A[0]) 
    row_1 = len(matrix_A) 
    col_2 = len(matrix_B[0]) 
    row_2 = len(matrix_B) 
 
    if (col_1 != row_2): 
        print("\nError: The number of columns in Matrix A  must be equal to the number of rows in Matrix B\n") 
        return 0
 
    # result_matrix_row = [0] * col_2
    result_matrix = [[0 for x in range(col_2)] for y in range(row_1)] 
 
    if (col_1 == 1): 
        result_matrix[0][0] = matrix_A[0][0] * matrix_B[0][0] 
 
    else: 
        split_index = col_1 // 2
 
        # row_vector = [0] * split_index 
        result_matrix_00 = [[0 for x in range(split_index)] for y in range(split_index)] 
        result_matrix_01 = [[0 for x in range(split_index)] for y in range(split_index)] 
        result_matrix_10 = [[0 for x in range(split_index)] for y in range(split_index)] 
        result_matrix_11 = [[0 for x in range(split_index)] for y in range(split_index)] 
        a00 = [[0 for x in range(split_index)] for y in range(split_index)] 
        a01 = [[0 for x in range(split_index)] for y in range(split_index)] 
        a10 = [[0 for x in range(split_index)] for y in range(split_index)] 
        a11 = [[0 for x in range(split_index)] for y in range(split_index)] 
        b00 = [[0 for x in range(split_index)] for y in range(split_index)] 
        b01 = [[0 for x in range(split_index)] for y in range(split_index)] 
        b10 = [[0 for x in range(split_index)] for y in range(split_index)] 
        b11 = [[0 for x in range(split_index)] for y in range(split_index)] 
 
        for i in range(split_index): 
            for j in range(split_index): 
                a00[i][j] = matrix_A[i][j] 
                a01[i][j] = matrix_A[i][j + split_index] 
                a10[i][j] = matrix_A[split_index + i][j] 
                a11[i][j] = matrix_A[i + split_index][j + split_index] 
                b00[i][j] = matrix_B[i][j] 
                b01[i][j] = matrix_B[i][j + split_index] 
                b10[i][j] = matrix_B[split_index + i][j] 
                b11[i][j] = matrix_B[i + split_index][j + split_index] 
 
        add_matrix(multiply_matrix(a00, b00),multiply_matrix(a01, b10),result_matrix_00, split_index)
        add_matrix(multiply_matrix(a00, b01),multiply_matrix(a01, b11),result_matrix_01, split_index)
        add_matrix(multiply_matrix(a10, b00),multiply_matrix(a11, b10),result_matrix_10, split_index)
        add_matrix(multiply_matrix(a10, b01),multiply_matrix(a11, b11),result_matrix_11, split_index)
 
        for i in range(split_index): 
            for j in range(split_index): 
                result_matrix[i][j] = result_matrix_00[i][j] 
                result_matrix[i][j + split_index] = result_matrix_01[i][j] 
                result_matrix[split_index + i][j] = result_matrix_10[i][j] 
                result_matrix[i + split_index][j + split_index] = result_matrix_11[i][j] 
 
    return result_matrix

## Frequency Count Method

In [11]:
X_y = []
i=5
for n in range(2,i+1):
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(multiply_matrix)
  
  # create matrices with dimension nxn
  matrix1, matrix2 = create_matrices(n)
  lp_wrapper(matrix1, matrix2)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [12]:
resultados_com_menor_loss = []
repeat = 4
registros = []
unary_operators_list = ["log", "square", "cube", "sqrt", "round", "exp", "abs"]

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=unary_operators_list
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)
    
sys.stdout = original_stdout



[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-5, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           5.293e+05  1.594e+01  y = x₀                                        2           5.126e+05  3.200e-02  y = square(x₀)                                3           1.919e+05  9.827e-01  y = square(square(x₀))                        5           1.391e+05  1.608e-01  y = square(square(x₀) * -1.127)               7           8.105e+04  2.701e-01  y = (square(square(x₀)) + square(square(x₀))) 9           6.492e+04  

In [13]:
registros_ = registros
file_pickle = 'dados_v2.pck'

In [14]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [15]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)

caminho_arquivo = file_pickle
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'strassen_N^2.81', caminho_arquivo)

In [16]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = file_pickle
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  print('\u25CF', k)
  for index, item in enumerate(v):
    if index == len(v)-1:
       print('└─', item)
    else:
      print('├─', item)
  print('==========================')

Conteúdo do arquivo pickle:
● binary_search_logx
├─ [6, 1.2914093, 1.37589542429269, log(x0**6) + 5.574131]
└─ [5, 1.5291592, 2.19479656667366, log(x0)**2 + 13.966276]
● boyer_moore_M_plus_N
├─ [9, 1729031400.0, 0.1972007458935316, 752018330.701125*(0.00603869219560025*x0 + 0.00603869219560025*x1 - 1)**4 + 601415.75]
└─ [5, 15754614000.0, 0.3055240605113849, 3621132.2 - 28185.828*x1]
● bubblesort_N^2
├─ [4, 5910.4966, 3.4645076652491285, 1.5253483*x0**2]
└─ [4, 8542.308, 3.303255717418494, 1.5552218*x0**2]
● fibonacci_interative_N
├─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
└─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
● fibonacci_recursive_2^N
├─ [15, 111113150.0, 0.37865996607381686, x0**4*(0.0022606994*x0**2 - 0.714783)]
└─ [15, 77371830.0, 0.38657145558895856, x0**2*(x0**2*(0.16452532*x0 - 3.8766947) + 297.9323)]
● heapsort_NlogN
├─ [9, 0.24989581, 1.1851428541960574, 4*x0 - 1.4897935]
└─ [9, 0.24989575, 1.185128414350167, 4*x0 - 1.4897884]
● insertionsort_N^2
├─ [4, 12